In [ ]:
# Import the required packages
from AlgorithmImports import *
import random
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

class BuffettFearAndGreedAlgorithm(QCAlgorithm):
    # Initialise the class
    def initialize(self) -> None:
        # Set the start and end dates
        self.set_start_date(2015, 8, 18)
        self.set_end_date(2025, 8, 18)
        # Allocate the cash used for the strategy
        self.set_cash(200000)
        # Set the random seed
        random.seed(self.get_parameter('seed', 30))
        # Assign a flag on whether the regime filter should be used or not (1 is True, 0 is False)
        self._filter = bool(self.get_parameter('filter', 1))
        # Then determine the probability that a trade is executed in each minute
        self._prob_of_trade_min = 0.005
        # Then add the fear and greed index data
        self._fr_gd_index = self.add_data(FearGreedIndex, 'FG')
        self._fr_gd_index.history = self.history(self._fr_gd_index.symbol, datetime(2015, 7, 1), self.time).loc[self._fr_gd_index.symbol].qcindex
        self._fr_gd_index.regime = None
        # Finally add the S&P 500 as the investment universe and chose the fee model
        self._sp500 = self.add_equity('SPY')
        self._sp500.set_fee_model(InteractiveBrokersFeeModel())
        # Then liquidate the positions at the end of each day's trading
        self.schedule.on(self.date_rules.every_day(self._sp500.symbol), self.time_rules.before_market_close(self._sp500.symbol, 0), self.liquidate)

    def on_data(self, data: Slice) -> None:
        if self._sp500.symbol in data and self._sp500.exchange.hours.is_open(self.time + timedelta(minutes=1), False):
            # First determine if a trade should be executed
            execute_trade = random.random() < self._prob_of_trade_min
            if not execute_trade:
                return
            # If the filter is on, then take a long position during the fear regime
            if self._filter and self._fr_gd_index.regime == 1:
                return
            self.market_order(self._sp500.symbol, 200 if not self.portfolio.invested else -200)
        # Then wait until the Fear and Greed Index has new data
        if self._fr_gd_index.symbol not in data:
            return
        self._fr_gd_index.history.loc[self.time] = self._fr_gd_index.close
        # Then fit the Markov model and determine if there has been a regime change
        regime = pd.Series(MarkovRegression(self._fr_gd_index.history, k_regimes=2).fit().smoothed_marginal_probabilities.values.argmax(axis=1), index=self._fr_gd_index.history.index)

        # Finally, we check to ensure that a regime of 0 is a fearful one and one of 1 is a greedy one
        self.plot('Regime mean', 'Fear', self._fr_gd_index.history[regime[regime == 0].index].mean())
        self.plot('Regime mean', 'Greed', self._fr_gd_index.history[regime[regime == 1].index].mean())
        # Finish by saving the current regime and plotting it
        self._fr_gd_index.regime = regime.iloc[-1]
        self.plot('Regime', 'Current', self._fr_gd_index.regime)